# Agentive Chatbot

In [67]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")

# separate this so we have separate semantic routes for categories and all subcategories
agent_dir = "agents"
all_routes = []
prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)

        # 1 Route object per agent
        route = Route(
            name=agent_data['agent_name'],
            utterances=agent_data['utterances']
        )
        all_routes.append(route)

        # Stores the agentive prompt in dictionary
        prompts[agent_data['agent_name']] = agent_data['prompt']

encoder = OpenAIEncoder()
question_rl = RouteLayer(encoder=encoder, routes=all_routes)

#### Variables to store the default system prompt and the catch-all prompt (to be used when no route is identified)

In [68]:
system_prompt = """
You are a woman in your early 30s thinking about purchasing health insurance. 
This is your first meeting with the insurance agent. You are having a conversation with the insurance agent.
"""

catchall_prompt = """
Your main goals for this conversation is to find out more about the insurance policies that the insurance agent sells, and also ascertain if the agent is trustworthy to work with.
"""

#### Defines logic for updating, activating, and deactivating prompts

In [69]:
#returns one route per input
def classify_question(prompt: str) -> str | list[str]:
    r = question_rl(prompt)
    return r.name

OFFENDED_BOOL = False
DISTANT_BOOL = False
DISTANT_TIMER = 0

def update_prompt(user_input: str) -> str:
    global OFFENDED_BOOL
    global DISTANT_BOOL
    global DISTANT_TIMER
    question_category = classify_question(user_input)
    print(f"route: {question_category}")
    prompt = ""
    if question_category == "offended":
        if not OFFENDED_BOOL:
            prompt = prompts["offended"]
            OFFENDED_BOOL = True
        else:
            prompt = prompts["enraged"]
    elif question_category == "distant" and OFFENDED_BOOL:
        prompt = prompts["distant"]
        OFFENDED_BOOL = False
        DISTANT_BOOL = True
        DISTANT_TIMER += 1
    elif DISTANT_TIMER > 5:
        prompt = catchall_prompt
        DISTANT_BOOL = False
    elif DISTANT_BOOL:
        prompt = prompts["distant"]
        DISTANT_TIMER += 1
    elif OFFENDED_BOOL:
        prompt = prompts["offended"]
    else:
        prompt = catchall_prompt
    return prompt

In [70]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []  # Store all conversations
    
    def query(self, query: str, prompt: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)  # Remove the role message for the next query
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                for message in conversation:
                    if message["role"] == "user":
                        file.write("User: " + message["content"] + "\n")
                    elif message["role"] == "assistant":
                        file.write("Agent: " + message["content"] + "\n")

rpa = ChatBot(api_key=api_key, role = system_prompt)

In [71]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    PROMPT = update_prompt(pasted_content)
    print(f"current prompt: {PROMPT}")
    rpa.query(pasted_content, prompt=PROMPT)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

# Additional input field for pasted content
pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')  # Adjust height as needed
)
# Button for pasted content submission
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'  # Change to your desired color

# Button for saving chat history
save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'  # Change to your desired color

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

# Arrange buttons side by side and center them using HBox
button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

# Display the input fields and the button layout
display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')